In [21]:
%pip install scikit-learn

from sklearn.preprocessing import LabelEncoder

class IntentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_encoder = LabelEncoder()
        self.dataframe['label'] = self.label_encoder.fit_transform(self.dataframe['intent'])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row['conversation']
        label = row['label']

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [23]:
# Load the training data
train_df = pd.read_csv('C:/Users/umesh.malviya1/Downloads/intent_classification_data.csv')





In [24]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Tokenize and create DataLoader
train_dataset = IntentDataset(train_df, tokenizer, max_length=32)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Prepare optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# Training loop
for epoch in range(10):
    print(f'Epoch {epoch + 1}/10')
    print('-' * 10)

    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f'Average Loss: {avg_loss}')

# Save the trained model
model.save_pretrained('C:/Users/umesh.malviya1/Downloads/intent_classifier')
tokenizer.save_pretrained('C:/Users/umesh.malviya1/Downloads/intent_classifier')

Epoch 1/10
----------
Average Loss: 0.9165508349736532
Epoch 2/10
----------
Average Loss: 0.49037498732407886
Epoch 3/10
----------
Average Loss: 0.2545113724966844
Epoch 4/10
----------
Average Loss: 0.10656096910436948
Epoch 5/10
----------
Average Loss: 0.0925748396354417
Epoch 6/10
----------
Average Loss: 0.030170060073335964
Epoch 7/10
----------
Average Loss: 0.020582498128836352
Epoch 8/10
----------
Average Loss: 0.01261688667970399
Epoch 9/10
----------
Average Loss: 0.009674558086165538
Epoch 10/10
----------
Average Loss: 0.00741783413104713


('C:/Users/umesh.malviya1/Downloads/intent_classifier\\tokenizer_config.json',
 'C:/Users/umesh.malviya1/Downloads/intent_classifier\\special_tokens_map.json',
 'C:/Users/umesh.malviya1/Downloads/intent_classifier\\vocab.txt',
 'C:/Users/umesh.malviya1/Downloads/intent_classifier\\added_tokens.json')

('C:/Users/umesh.malviya1/Downloads/intent_classifier\\tokenizer_config.json',
 'C:/Users/umesh.malviya1/Downloads/intent_classifier\\special_tokens_map.json',
 'C:/Users/umesh.malviya1/Downloads/intent_classifier\\vocab.txt',
 'C:/Users/umesh.malviya1/Downloads/intent_classifier\\added_tokens.json')